### Here XRD data are visualized and analyzed 

In [ ]:
from functions import *

sample = "mittma_0022_FR"
#path to the Data / Sample folder
folder = r"Z:\P110143-phosphosulfides-Andrea\Data\Samples\mittma_0022_Cu\XRD" 

#### Raw data treatment

In [ ]:
grid = measurement_grid(5,5,34,34,-17,3) # BR_map 3mm margin (back sample)
# grid = measurement_grid(5,10,32,72,-16,-40) # 2 samples

filename = "mittma_0022_FR_map.txt" 
filename = os.path.join(folder, filename)

initial_data, coords = read_XRD(filename, grid, n = 0, separator = "\t")
plot_grid(coords,grid)

Raw data visualization and selection or translation

In [ ]:
datatype_y= 'Intensity, cps'
datatype_x='2θ (°)'
plot_data(initial_data, datatype_x, datatype_y , plotscale = 'log')
display(initial_data)

# export_specific(initial_data, ['2θ (°)', 'Intensity, cps'], 17.0, 17.0, path=os.path.join(exp_path, sample+"_17_17.txt"))

In [ ]:
# if you want to work only on a limited number of points 
which =initial_data.keys()[0:8]
data= initial_data[which]
display(data)

In [ ]:
# translate data if needed
data, coords = translate_data(initial_data, x=0, y=-20)
X,Y= extract_coordinates(data)
display(pd.DataFrame({'X':X, 'Y':Y}))
# MI_to_grid(data) #prints new coordinates

Remove the background and save processed data for future use

In [ ]:
# display(initial_data)
# data=initial_data # uncomment this if you use the whole data, withour any selection or translation
print(data.columns)

In [ ]:
#cut data to the region of interest , if needed

last_2theta= 60

col_theta = data.columns[data.columns.get_level_values(1)=='2θ (°)']
col_counts= data.columns[data.columns.get_level_values(1)=='Intensity, cps']
coords= data.columns.get_level_values(0).unique()
# print(col_theta)
columns= data.columns.get_level_values(1).unique()
for i in range(0,len(coords)):
    # print(data.columns.get_level_values(0).unique()[i])
    print(coords[i])
    # print(data[col_theta[i]])
    idx_last = np.where(data[col_theta[i]]>last_2theta)[0][0]
    print(idx_last)

data.drop(data.index[idx_last:], inplace=True)

display(data)

In [ ]:
#  preliminary finidng peaks 
dataRangeMin = 0
dataRangeMax = len(data)
filterstrength = 10 # higher number = more noise is removed by the filter
peakprominence = 100 # higher number = only peaks with prominence above this value are considered
peakwidth = 20
 # higher number = wider peaks are considered

peaks, dataCorrected = initial_peaks(data, dataRangeMin, dataRangeMax, filterstrength,
                                         peakprominence, peakwidth,
                                         withplots = True, plotscale = 'linear')

In [ ]:
#  remove the background 
data_out = XRD_background(data,peaks, cut_range=1, order=4, withplots= True, Si_cut=True)

In [ ]:
#  save clean data for future use 

name = sample + "_clean.pkl"
# name = "mittma_0019_R_XRD.pkl"

with (open(os.path.join(folder, name), "wb")) as openfile:
    pickle.dump(data_out, openfile)

### Analysis section

In [19]:
## load clean data, if previously processed  
# folder=r"Z:\P110143-phosphosulfides-Andrea\Data\Samples\mittma_0015_Cu\XRD"

# name = sample + "_XRD.pkl"
# # name= sample +"_clean.pkl"
# with open(os.path.join(folder, name), "rb") as openfile:
#     data_out = pickle.load(openfile) 

##  load refence spectra

ref_path = r"Z:\P110143-phosphosulfides-Andrea\Data\Analysis\guidal\XRD\ref_database\reflections"

with open(os.path.join(ref_path, "reflections.pkl"), 'rb') as f:
    ref_peaks_df = pickle.load(f)

display(ref_peaks_df["Cu3PS4"].head(10))
# display(data_out)

,Peak 2theta,Peak intensity,2theta,I
0,28.228228,530.563623,20.000000,1.142195
1,31.891892,481.050563,20.060060,3.660634
2,53.693694,361.288970,20.120120,8.179245
3,29.369369,332.629103,20.180180,12.741217
4,49.909910,262.777901,20.240240,13.837239
5,41.141141,164.261586,20.300300,10.476803
6,58.978979,143.929880,20.360360,5.530304
7,76.396396,71.074564,20.420420,2.035213
8,60.540541,37.644476,20.480480,0.522170
9,NaN,NaN,20.540541,0.093402


#### Interactive plot to find possible phases

In [ ]:
x, y = extract_coordinates(data_out)
# x,y = select_points(data_out, y_max=0)
print(x)
print(y) 

last_2theta= 60
data=data_out

col_theta = data.columns[data.columns.get_level_values(1)=='2θ (°)']
# col_counts= data.columns[data.columns.get_level_values(1)=='Intensity, cps']
coords= data.columns.get_level_values(0).unique()
# print(col_theta)
columns= data.columns.get_level_values(1).unique()
for i in range(0,len(coords)):
    # print(data.columns.get_level_values(0).unique()[i])
    print(coords[i])
    # print(data[col_theta[i]])
    idx_last = np.where(data[col_theta[i]]>last_2theta)[0][0]
    print(idx_last)
    # data[col_theta[i]]= data[col_theta[i]].iloc[:idx_last]
    # data[col_counts[i]]= data[col_counts[i]].iloc[:idx_last]

data.drop(data.index[idx_last:], inplace=True)

display(data)

# plot_data(data_out, '2θ (°)', 'Intensity, cps',x=-18,y=18, plotscale = 'log')

In [ ]:
# data_shifted, coords= translate_data(data, x=20, y=0)

plot_grid(coords,coords)

In [ ]:
# plot all the points in the interactive plot
x, y = extract_coordinates(data_out)

savepath= r"Z:\P110143-phosphosulfides-Andrea\Data\Analysis\guidal\XRD" # change to your path
savepath = os.path.join(savepath, sample+"_interactive.html")

fig= interactive_XRD_shift(data_out, '2θ (°)', 'Corrected Intensity', 400, x,y, ref_peaks_df, 
                           title= sample +' XRD map',
                        #    savepath=savepath
                           )


In [ ]:
# make specific plots
x,y= select_points(data_out, x_min=10)
plot_XRD_shift(data, '2θ (°)', 'Corrected Intensity', 600, x,y, savepath=False, show=False)

refs= ['Cu2S', 'CuS2',
       'Cu7S4', 'Cu9S5', 
       #'Cu3S2'
       ]
shift = 1000
dampen= ['Cu9S5', 'Cu3S2']
coeff=2
colors = ['brown', 'g', 'b', 'pink']
i=0
for ref in refs:
    if ref in dampen:
        coeff=0.5

    plt.plot(ref_peaks_df[ref]['2theta'], ref_peaks_df[ref]['I']*coeff-shift, color=colors[i], label=ref)
    plt.vlines(ref_peaks_df[ref]['Peak 2theta'],-3000, 3000, color=colors[i], linestyles='dashed', alpha=0.5)
    shift+= 1000
    i+=1

plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
plt.xlim(20, 60)
plt.show()

In [ ]:
#this is just a box for notes on the phases you see
phase_A = [5,10]
phase_B = [23,24]
phase_C = [8,9,13,14,15,17, 18,19,20,22]
phase_D = [3,4,11,12,16,21]
phase_E = [1,2,6,7]
Cu8P2S = [25] 

B= list(np.linspace(21,30,10).astype(int))
A= np.linspace(41,50,10)
print(B)
print(A)

#### export specific data to a txt file

In [ ]:

exp_path = r"Z:\P110143-phosphosulfides-Andrea\Data\Analysis\guidal\XRD"
export_specific(data_out, ['2θ (°)', 'Corrected Intensity'], 17.0, 17.0, path=os.path.join(exp_path, sample+"_17_17_sulphide.txt"))


#### Assign phases to different points

In [ ]:
phase_info = assign_phases_labels(data_out)

In [ ]:
new_df = add_info(data_out,phase_info)
display(new_df) #check that phases are assigned properly

In [ ]:
# save the phases df in a pickle file (not necessary probably)
pkl_path = r"Z:\P110143-phosphosulfides-Andrea\Data\Analysis\guidal\XRD" 
with open(os.path.join(folder, sample+"new_phases.pkl"), "wb") as openfile:
    pickle.dump(new_df, openfile)

#### Fit specific peaks 

In [ ]:
## remove unwanted datapoints, or columns
# data_to_analyze = data_out.drop(columns=data_out.columns[data_out.columns.get_level_values(0) == '18.0,-18.0'])
# data_to_analyze = data_to_analyze.drop(columns=data_to_analyze.columns[data_to_analyze.columns.get_level_values(1) == 'Intensity, cps'])
# data_to_analyze = data_to_analyze.drop(columns=data_to_analyze.columns[data_to_analyze.columns.get_level_values(1) == 'Background'])
# which =data_out.keys()[0:100]
# data_to_analyze= data_out[which]
# display(data_to_analyze)
data_to_analyze= data_out # if you want to work on the whole dataset

data_to_analyze = data_to_analyze.drop(columns=data_to_analyze.columns[data_to_analyze.columns.get_level_values(1) == 'Intensity, cps'])
data_to_analyze = data_to_analyze.drop(columns=data_to_analyze.columns[data_to_analyze.columns.get_level_values(1) == 'Background'])
print(data_to_analyze.columns.get_level_values(1).unique())
print(data_to_analyze.columns.get_level_values(0).unique())

In [ ]:
new_peaks, new_data = initial_peaks(data_to_analyze, 0, len(data_to_analyze),
                                    filterstrength= 15,
                                    peakprominence= 90,
                                    peakheight= 110,
                                    peakwidth= 20,
                                    withplots=True, plotscale='linear')

In [ ]:
display(new_peaks)
display(ref_peaks_df['Cu7PS6'].sort_values(by='Peak 2theta'))

In [ ]:
# define peak position to analyze, and fit the peak with two pseudovoigts (ka1, ka2)

peak_position = 32.05
df_first_peak = fit_this_peak(data_to_analyze, peak_position, 0.7, withplots = True, printinfo = False)
# save it manually if you want to keep it

In [ ]:
ref= 32.072072 # peak position of the reference peak
df_first_peak = math_on_columns(df_first_peak, 'Center', ref, '-')
df_first_peak.rename(columns={f'Center - {ref}': 'shift'}, inplace=True)
display(df_first_peak.head(4))

In [ ]:
#  plot info from peak fitting in a heatmap

list_to_plot = ["Center", "Amplitude", "FWHM", "shift"]#, "Fraction"]
plots_path = r"Z:\P110143-phosphosulfides-Andrea\Data\Analysis\guidal\XRD\Cu7PS6" # change to your path

# list_to_exclude = [[18, 0], [18,8]]
for item in list_to_plot:
    savepath = os.path.join(plots_path, f'{sample}_{peak_position}_{item}.png')
    title= 'Cu<sub>7</sub>PS<sub>6</sub> without S-Cracker - fit at 2θ= ' +f'{peak_position}° - {item}'
  
    new_heatmap(item, data= df_first_peak, title =  title, 
                # exclude= list_to_exclude,
                savepath = savepath
                )

In [ ]:
# if you want to look at specific point in the dataset you can call them with plot_data
plt.figure(figsize=(7, 5))
x=[0,0]
y=[20,-20]

plot_data(df_first_peak, 'range 2θ', 'Fit', x,y, plotscale = 'linear')
plot_data(df_first_peak, 'range 2θ', 'range Intensity',x,y, legend =False, scatter_plot=False,  plotscale = 'linear' )
#plt.savefig(os.path.join(plots_path, "mittma_00015_FR_second_peak_weird points.png"))

In [ ]:
# ------------------- calculate shift from reference -------------------

ref_peak_pos = 27.807808

data= df_first_peak.copy()
dat_center = data.iloc[:,data.columns.get_level_values(1)=='Center']
dat_center = dat_center.dropna()
shift = dat_center.values - ref_peak_pos

data = math_on_columns(data, 'Center', type2= ref_peak_pos, operation = "-")
data.rename(columns={f'Center - {ref_peak_pos}':'Center - Reference'}, inplace = True)

In [ ]:
# calculate spacing 
lambda_k_alpha1 = 1.54059

theta_min=np.radians(45.3/2)
theta_max=np.radians(45.7/2)
d_calc_min= lambda_k_alpha1/(2*np.sin(theta_min))
d_calc_max= lambda_k_alpha1/(2*np.sin(theta_max))
d_ref = lambda_k_alpha1/(2*np.sin(np.radians(45.1051/2)))
print(d_ref)
print(d_calc_min,d_calc_max)
print(100*(d_ref-d_calc_min)/d_ref, 100*(d_ref-d_calc_max)/d_ref)

### Integrate with EDX data for the same sample

In [ ]:
sample_id = 'mittma_0019' # name of the file to pick up is sample_id_EDS.pkl
quarter = 'R' # 

EDS_pickle_path = r"Z:\P110143-phosphosulfides-Andrea\Data\Analysis\guidal\mittma_pickles"
with open(os.path.join(EDS_pickle_path, f"{sample_id}_EDS.pkl"), "rb") as openfile:
    EDS_data = pickle.load(openfile)

display(EDS_data)
grid = MI_to_grid(EDS_data)
plot_grid(grid,grid)
x,y= select_points(EDS_data, x_min=0, y_max=-30)
print(x,y)

In [ ]:
if quarter == 'R':
    new_x, new_y = select_points(EDS_data, x_min=-10)
if quarter == 'L':
    new_x, new_y = select_points(EDS_data, x_max =10)
if quarter == 'BR':
    new_x, new_y = select_points(EDS_data, y_min=-10, x_min=-11, x_max =40, y_max=40)
if quarter == 'FR':
    new_x, new_y = select_points(EDS_data, y_max=10, x_min=-10)

new_coords = list(('{},{}').format(x,y) for x,y in zip(new_x, new_y))
#display(new_coords)

# EDS_FR, coords = translate_data(EDS_data[new_coords], x=-17, y=17) # if you want to translate the data to the coordinates of 1 square
EDS_FR = EDS_data[new_coords] # if you want to keep the full sample coordinates
display(EDS_FR)
grid = MI_to_grid(EDS_FR)
plot_grid(grid,grid)

In [ ]:
#interpolate EDS data to the same grid as XRD data

X,Y= extract_coordinates(data_out) # use the df from the XRD data you want to join
grid = pd.DataFrame({'x':X, 'y':Y})
EDS_interp = interpolate_grid(EDS_FR, grid)
# print(EDS_interp.columns.get_level_values(0).unique())
display(EDS_interp)


In [ ]:
# Find columns (coordinates) where any value is NaN and assign value from closest point
nan_coords = [col for col in EDS_interp.columns.levels[0] if EDS_interp[col].isnull().any().any()]
other_coords = [col for col in EDS_interp.columns.levels[0] if col not in nan_coords]

x_grid, y_grid= extract_coordinates(EDS_interp[other_coords])
EDS_grid = pd.DataFrame({'x':x_grid, 'y':y_grid})
for col in nan_coords:
    x = float(col.split(',')[0])
    y = float(col.split(',')[1])
    closest_x, closest_y = closest_coord(EDS_grid, x, y)
    print(f'missing data at {col}, closest point is {closest_x},{closest_y}')
    EDS_interp[col] = EDS_interp[f'{closest_x},{closest_y}']

info_array=[]

for col in EDS_interp.columns.get_level_values(0).unique():
    if col in nan_coords:
        info_array.append('assigned from closest point')
    elif col not in nan_coords:
        info_array.append('measured')
info_composition= {'composition': info_array}

EDS_interp = add_info(EDS_interp, info_composition)
display(get_data(EDS_interp, 'composition'))

# heatmap( data= EDS_interp, datatype= "Layer 1 Cu Atomic %")
new_heatmap('Layer 1 Cu Atomic %', EDS_interp)

In [ ]:
# check if the data is correctly interpolated
get_data(EDS_interp, x=8.5, y=8.5)

In [ ]:
#combine the data

data_combined = combine_data((new_df, EDS_interp))
#to check if the data is combined correctly
display(data_combined)
# display(get_data(data_combined, x=3, y=20))

#### pickle checkpoint

In [ ]:
# save the result in a pickle file, to be used in the ternary plot
XRD_pickles_path = r"Z:\P110143-phosphosulfides-Andrea\Data\Analysis\guidal\XRD\pickles"

with open(os.path.join(XRD_pickles_path, sample+"new_phases_coords.pkl"), "wb") as openfile:
    pickle.dump(data_combined, openfile)

# sample = "mittma_0023_FR"
# with open(os.path.join(XRD_pickles_path, sample+"new_phases_coords.pkl"), "rb") as f: #new_phases_coords
#     data_combined= pickle.load(f)
# display(data_combined)

In [ ]:
# plot phases distribution on the sample map
x,y = extract_coordinates(data_combined)
labels= get_data(data_combined, 'Phase').loc[0].values

df_to_plot= pd.DataFrame({'X':x, 'Y':y, 'Phase':labels})
import plotly.express as px
# title= 'Phases in A-5 mTorr, S-Cracker On - right samples'
title= f'Phases in {sample}'
fig= px.scatter(df_to_plot, x='X', y='Y', color='Phase', title= title,
                width=500, height=400)
fig.update_traces(marker=dict(size=20))

# fig.write_image(os.path.join(XRD_pickles_path, sample+"_phases_coords.png"), scale=2)
fig.show()


#### plot in a ternary plot

In [ ]:
sample= 'Cu-rich'
title = f'Phases in {sample} samples'

# title= 'Phases in B-15 mTorr, S-Cracker Off - bottom right sample'
savepath =os.path.join(XRD_pickles_path, f'{sample}_phases_ternary.html')
ternary_discrete(all_B, 'Cu', 'P', 'S', intensity_label='Phase',shape_label='Phase', #composition
                         title=title, 
                         savepath=savepath,
                        )